In [35]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [36]:
dossier="/Github/eGloss"
enonce="Exo1"
theme="GlosesMots"
adaptatifPenalite="0.1000000"
dernierIndiceNote=75.0
msgPlusDeMots=u"Il y a plus de mots"
msgMoinsDeMots=u"Il y a moins de mots"
def nbIndiceNotes(nb):
    if nb==1: 
        return ["%50%"]
    elif nb==2:
        return ["","%50%"]
    else:
        return [""]+["%"+str(int((i+2)*dernierIndiceNote/nb))+"%" for i in range(nb-1)]   

In [37]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [38]:
from MoodleXML import *
import yaml

In [39]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=True
titreNoEX=False

In [40]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r', encoding="utf8") as file:
    donneesLignes=file.readlines()

In [41]:
def donneesEX():
    titre=[]
    result={}
    boolMultiLignes=False
    i=0
    for ligne in donneesLignes:
        if debug or 0: print ligne
        i+=1
        if ligne.startswith("#EX;"):
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#EXM;"):
            boolMultiLignes=True
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                titre=temp[1:]            
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            if boolMultiLignes:
                result["boucle"].extend(temp)
            else:
                result["boucle"].append(temp)
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
    donneesEX()
else:
    donneesNoEX()

In [42]:
def lierMot(ref,chaine):
    if motReferences[ref]:
        mot=motReferences[ref]
    else:
        mot=ref
    result=chaine.replace(ref,u'<a href="https://fr.wikipedia.org/wiki/%s" target="_blank" style="color:blue">%s</a>'%(mot,ref))
    return result

In [43]:
def modifierCorrection(correction):
    result=correction
    subPoints = re.compile(ur"(\.(?![*+])|-(?![-.])|_)")
    result=subPoints.sub(ur"\S*",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()        
        if re.search(ur"(\(([^)]*)\)|\[([^]]*)\])",corps):
            boolIntermediaire=True
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\S*\1\S*",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\S*\1\S*",corps)
        unites=filter(None,corps.split("&&"))
#        print "unités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
    return result    

def echapperCorrection(correction):
    boolIntermediaire=False
    listeResult=[]
    subComplexe=re.compile(ur"(\.(?![*+])|-(?![-.])|_)")
    if subComplexe.search(correction):
        boolIntermediaire=True
    result=correction
#    print "original",result
    subPoints = re.compile(ur"(\.(?![*+]))")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()        
        subParentheses = re.compile(ur"\(([^)]*)\)")
        if subParentheses.search(corps):
            boolIntermediaire=True
            corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        if subCrochets.search(corps):
            boolIntermediaire=True
            corps=subCrochets.sub(ur"\\[\1\\]",corps)
        unites=filter(None,corps.split("&&"))
#        print "unités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
    if boolIntermediaire:
        print "intermédiaire",correction
        listeResult.append(modifierCorrection(correction))
    listeResult.append(result)
#    print "échappé",result
    return listeResult    

In [44]:
def separerMots(elements,titre,reponse):
    result=[]
    elements = filter(None, elements)
    nbMots=len(reponse.strip().split(" "))
    solution=u"=%s#Bravo ! La phrase était : %s"%(reponse,titre)
    corrections=[]
    nbElements=len(elements[1:])
    for n,element in enumerate(elements[1:]):
        (reponseSpecifique,feedbackSpecifique)=element.split("#",1)
        feedbackSpecs=feedbackSpecifique.split("#")
        reponseSpecs=echapperCorrection(reponseSpecifique)
#        print "echapperCorrection", reponseSpecs
        for i,reponseSpec in enumerate(reponseSpecs):
            print i,reponseSpec
            if i <len(feedbackSpecs):
                feedbackSpec=feedbackSpecs[i]
            else:
                feedbackSpec=feedbackSpecifique
            print "feedback", feedbackSpec
            for mot in motReferences:
                if mot in feedbackSpec:
                    feedbackSpec=lierMot(mot,feedbackSpec)
            element="#".join([reponseSpec,feedbackSpec])
            corrections.append(nbIndiceNotes(nbElements)[n]+element)
    minMots="^"+"([^ ]+ +)?"*(nbMots-4)+"[^ ]+$#%s"%msgPlusDeMots
    maxMots="^"+"([^ ]+ +)"*(nbMots+2)+"([^ ]+ +)*[^ ]+$#%s"%msgMoinsDeMots
    result.extend([solution,minMots,maxMots])
    result.extend(corrections)
    return result

In [45]:
consigne="<br/>".join(configuration[u"Consigne"])
motReferences=configuration[u"Références"]
separationExercices=[]
for question in donnees:
    questionLignes=[]
    for nom in question:
        titre=nom
        questionLignes.append(consigne)
        if "#" in question[nom]["boucle"][0]:
            (phraseRep,phraseQuest)=question[nom]["boucle"][0].split("#",1)            
        else:
            phraseRep=question[nom]["boucle"][0]
            phraseQuest=phraseRep
#        print "réponse", phraseRep
#        print "question", phraseQuest
        questionLigne=u"".join(re.split("[ -]",phraseQuest))
        questionLignes.append(u"".join(re.split(u"[ -]",questionLigne)))
#        questionLignes.append(u"".join(question[nom]["boucle"][0].split(" ")))
        questionLignes.append(u"{1:RXC:%s}"%u"~".join(separerMots(question[nom]["boucle"],titre,phraseRep)))
#        print questionLignes
        textQuestion=u"<br/>".join(questionLignes)
        separationExercices.append(ClozeExercice(titre,textQuestion,penalty=adaptatifPenalite))

0 --.*( chat.* gris.* souris.*observ.* haut.* gouttière.* vert.* clair).*
feedback Il manque des mots lexicaux.
0 --.*(DEF.* DE DEF).*
feedback Il manque des mots grammaticaux (à noter en majuscules).
intermédiaire --.*(&& chat(M)[SG] && souris(F)[SG] && haut(M)[SG] && gouttière(F)[SG] ).*
0 --.*( chat\S*M\S*\S*SG\S* .* souris\S*F\S*\S*SG\S* .* haut\S*M\S*\S*SG\S* .* gouttière\S*F\S*\S*SG\S* ).*
feedback La glose des noms comprend leur genre et leur nombre.
1 --.*( chat\(M\)\[SG\] .* souris\(F\)\[SG\] .* haut\(M\)\[SG\] .* gouttière\(F\)\[SG\] ).*
feedback La glose des noms comprend leur genre lexical et leur nombre en contexte.
intermédiaire --.*(&&DEF-M.SG.*chat&& DEF-F.SG.*gouttière).*
0 --.*(DEF\S*M\S*SG.*chat.* DEF\S*F\S*SG.*gouttière).*
feedback La glose des définis comprend leur genre et leur nombre.
1 --.*(DEF-M\.SG.*chat.* DEF-F\.SG.*gouttière).*
feedback Pour les définis le genre et le nombre constituent un suffixe.
intermédiaire --.*(&& gris(M)[SG] && vert(M)[SG] clair[M.SG]

In [46]:
categorie=configuration[u"Catégorie"]
clozeQuestions=XMLClozes(categorie)

In [47]:
for exercice in separationExercices:
    clozeQuestions.addExercice(exercice)

In [48]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(clozeQuestions.getClozes())

In [49]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [50]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [51]:
motReferences

{'clitique': None,
 'datif': None,
 u'd\xe9monstratif': u'Adjectif_d\xe9monstratif',
 'liaison': u'Liaison_en_fran\xe7ais',
 'partitif': 'Article_partitif',
 'possessif': None,
 u'pr\xe9position': None,
 u'r\xe9fl\xe9chi': u'Morphologie_du_pronom_personnel_en_fran\xe7ais#Pronom_r.C3.A9fl.C3.A9chi_et_pronom_non_r.C3.A9fl.C3.A9chi'}

In [52]:
correction=u"--.*(&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]).*"
#correction=u"--.*(&& DEF-M.SG.⊖ chat&& DEF-F.SG.⊖ gouttière).*"
(prefixe, corps, suffixe)=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",correction).groups()
print corps
subParentheses = re.compile(ur"\(([^)]*)\)")
result=subParentheses.sub(ur"\\(\1\\)",corps)
print result
subCrochets = re.compile(ur"\[([^]]*)\]")
result=subCrochets.sub(ur"\\[\1\\]",result)
subPoints = re.compile(ur"(\.[^*])")
result=subPoints.sub(ur"\\\1",result)

print prefixe+result+suffixe

&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]
&& gris\(M\)[SG.⊖] && vert\(M\)[SG.⊖] && clair[M.SG.⊖]
--.*(&& gris\(M\)\[SG\.⊖\] && vert\(M\)\[SG\.⊖\] && clair\[M\.SG\.⊖\]).*


In [53]:
import re
test=".*"
subPoints = re.compile(ur"(\.(?![*+]))")
if subPoints.search(test):
    print "found"
else:
    print "none"

none


In [54]:
chaine="--.*(&& chat(M)[SG]).*"
for element in echapperCorrection(chaine):
    print element

intermédiaire --.*(&& chat(M)[SG]).*
--.*( chat\S*M\S*\S*SG\S*).*
--.*( chat\(M\)\[SG\]).*


In [55]:
if "#" in question[nom]["boucle"][0]:
    print u"".join(re.split("[ -]",question[nom]["boucle"][0].split("#")[1]))
else:
    print u"".join(re.split("[ -]",question[nom]["boucle"][0]))

dəlasosɛtɔ̃bedəlaluʃsyʁseʃosyʁ
